In [ ]:
import requests
from IPython.display import display

all_products = []
page = 1
page_size = 100
target_records = 12000

while len(all_products) < target_records:
    url = f"https://world.openfoodfacts.org/api/v2/search?categories=chocolates&fields=code,product_name,brands,nutriments&page_size={page_size}&page={page}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()

        if not data.get('products'):
            print(f"No products found on page {page}. Stopping.")
            break # No more products

        all_products.extend(data['products'])
        print(f"Collected {len(all_products)} records so far.")
        page += 1

    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {page}: {e}")
        break # Stop if there's an error


print(f"Finished collecting {len(all_products)} records.")
display(all_products)

In [ ]:
len(all_products)

In [ ]:
import pickle

# Assuming 'all_products' contains your collected data
with open('all_products.pkl', 'wb') as f:
    pickle.dump(all_products, f)


In [ ]:
import pickle

with open('all_products.pkl', 'rb') as f:
    all_products = pickle.load(f)


In [ ]:
df=all_products.copy()

In [ ]:
display(df)

In [ ]:
import pandas as pd

In [ ]:
df=pd.DataFrame(all_products)

In [ ]:
df

In [ ]:
df1 = df.copy()

In [ ]:
df1

In [ ]:
import pandas as pd

In [ ]:
# Assuming your DataFrame 'df' is already loaded with data
# We'll create a new DataFrame for the selected nutrient data

# First, define the list of column names you want
columns_to_extract = [
    'energy-kcal_value',
    'energy-kj_value',
    'carbohydrates_value',
    'sugars_value',
    'fat_value',
    'saturated-fat_value',
    'proteins_value',
    'fiber_value',
    'salt_value',
    'sodium_value',
    'nova-group',
    'nutrition-score-fr',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g'
]

# This is the syntax to flatten the 'nutriments' dictionary and select your columns
nutrients_df = df1['nutriments'].apply(pd.Series)[columns_to_extract]

# Optional: Combine this with your original DataFrame to keep columns like 'code' and 'product_name'
df1 = pd.concat([df1[['code', 'product_name','brands']], nutrients_df], axis=1)

# Now you can view the new DataFrame with only the columns you need
print(df1.head())

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
df1['sugar_to_carb_ratio'] = df1['sugars_value'] / df1['carbohydrates_value']

# Optional: Handle division by zero or missing values
df1['sugar_to_carb_ratio'] = df1['sugar_to_carb_ratio'].replace([float('inf'), -float('inf')], pd.NA)
df1['sugar_to_carb_ratio'] = df1['sugar_to_carb_ratio'].fillna(0)


In [ ]:
df1["sugar_to_carb_ratio"]

In [ ]:
import pandas as pd

# Define thresholds for calorie categories (adjust as needed)
low_threshold = 100
high_threshold = 300

# Assuming df is your DataFrame
def classify_calorie(kcal):
    if pd.isna(kcal):
        return None
    elif kcal < low_threshold:
        return 'Low'
    elif kcal <= high_threshold:
        return 'Moderate'
    else:
        return 'High'

df1['calorie_category'] = df1['energy-kcal_value'].apply(classify_calorie)


In [ ]:
df1["calorie_category"]

In [ ]:
import pandas as pd

# Define sugar content thresholds (adjust as needed)
low_sugar_threshold = 5.0
high_sugar_threshold = 15.0

def classify_sugar(sugar):
    if pd.isna(sugar):
        return None
    elif sugar < low_sugar_threshold:
        return 'Low Sugar'
    elif sugar <= high_sugar_threshold:
        return 'Moderate Sugar'
    else:
        return 'High Sugar'

df1['sugar_category'] = df1['sugars_value'].apply(classify_sugar)


In [ ]:
df1["sugar_category"]

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
df1['is_ultra_processed'] = df1['nova-group'].apply(lambda x: 'Yes' if x == 4 else 'No')


In [ ]:
df1["is_ultra_processed"]

In [ ]:
df1    


In [ ]:
kcal_90_percentile = df1['energy-kcal_value'].quantile(0.90)
df1 = df1[df1['energy-kcal_value'] <= kcal_90_percentile]



In [ ]:
df1

In [ ]:
csv = df1.to_csv("chococrunch.csv")=>//

In [ ]:
# Assuming df is your DataFrame
df2 = df1[['code','product_name','brands','energy-kcal_value', 'sugars_value', 'carbohydrates_value', 'sugar_to_carb_ratio']]


In [ ]:
df2

In [ ]:
df2.describe()

In [ ]:
df2.isnull().sum()

In [ ]:
df2['sugars_value'].fillna(df2['sugars_value'].mean(), inplace=True)
df2['carbohydrates_value'].fillna(df2['carbohydrates_value'].mean(), inplace=True)


In [ ]:
df2.isnull().sum()

In [ ]:
df2

In [ ]:
csv = df2.to_csv("chococrunch.csv")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for plots
sns.set(style="whitegrid")

# 1. Check the shape and completeness of the dataset
#print("Data shape:", df1.shape)
#print("Missing values per column:\n", df1.isnull().sum())


In [ ]:
# 2. Distribution of key numeric variables with histograms
fig, axs = plt.subplots(2, 2, figsize=(14, 10))
sns.histplot(df1['energy-kcal_value'], bins=30, kde=True, color='chocolate', ax=axs[0,0])
axs[0,0].set_title("Calories Distribution")

sns.histplot(df1['sugars_value'], bins=30, kde=True, color='brown', ax=axs[0,1])
axs[0,1].set_title("Sugars Distribution")

sns.histplot(df1['carbohydrates_value'], bins=30, kde=True, color='darkred', ax=axs[1,0])
axs[1,0].set_title("Carbohydrates Distribution")

sns.histplot(df1['sugar_to_carb_ratio'], bins=30, kde=True, color='sienna', ax=axs[1,1])
axs[1,1].set_title("Sugar to Carb Ratio Distribution")

plt.tight_layout()
plt.show()

In [ ]:
# 3. Bar charts for calorie_category and sugar_category counts
plt.figure(figsize=(12,5))
sns.countplot(data=df1, x='calorie_category', order=['Low','Moderate','High'], palette='ch:s=.25,rot=-.25')
plt.title("Number of Products by Calorie Category")
plt.show()

plt.figure(figsize=(12,5))
sns.countplot(data=df1, x='sugar_category', order=['Low Sugar','Moderate Sugar','High Sugar'], palette='dark:#5A9_r')
plt.title("Number of Products by Sugar Category")
plt.show()

In [ ]:
# 4. Pie chart for NOVA group distribution
nova_counts = df1['nova-group'].value_counts(dropna=False).sort_index()
plt.figure(figsize=(7,7))
plt.pie(nova_counts, labels=nova_counts.index, autopct='%1.1f%%', colors=sns.color_palette('pastel'))
plt.title("Product Distribution by NOVA Group")
plt.show()


In [ ]:
# 5. Compare Ultra-processed vs Minimally processed
plt.figure(figsize=(7,5))
sns.countplot(data=df1, x='is_ultra_processed', palette=['#8B4513','#D2691E'])
plt.title("Ultra-processed vs Minimally Processed Chocolate Products")
plt.show()

In [ ]:
# 6. Scatter plot: Calories vs Sugars colored by Ultra-processed flag
plt.figure(figsize=(10,6))
sns.scatterplot(data=df1, x='energy-kcal_value', y='sugars_value', hue='is_ultra_processed', palette=['green','red'], alpha=0.6)
plt.title("Calories vs Sugars by Ultra-processed Flag")
plt.xlabel("Calories per 100g")
plt.ylabel("Sugars per 100g")
plt.show()

In [ ]:
# 7. Heatmap of correlations between numeric nutrition variables
nutrition_cols = [
    'energy-kcal_value', 'energy-kj_value', 'carbohydrates_value', 'sugars_value',
    'fat_value', 'saturated-fat_value', 'proteins_value', 'fiber_value',
    'salt_value', 'sodium_value', 'nutrition-score-fr', 'fruits-vegetables-nuts-estimate-from-ingredients_100g'
]

plt.figure(figsize=(10,8))
corr = df1[nutrition_cols].corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True)
plt.title("Nutrient Correlation Heatmap")
plt.show()

In [ ]:
# 8. Boxplots of calories and sugars grouped by brands (top 10 brands by product count to keep plot readable)
top_brands = df1['brands'].value_counts().nlargest(10).index
plt.figure(figsize=(14,6))
sns.boxplot(data=df1[df1['brands'].isin(top_brands)], x='brands', y='energy-kcal_value')
plt.xticks(rotation=45)
plt.title("Calories Distribution for Top 10 Brands")
plt.show()

plt.figure(figsize=(14,6))
sns.boxplot(data=df1[df1['brands'].isin(top_brands)], x='brands', y='sugars_value')
plt.xticks(rotation=45)
plt.title("Sugars Distribution for Top 10 Brands")
plt.show()

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np

def get_connection():
    return mysql.connector.connect(
       host='gateway01.ap-southeast-1.prod.aws.tidbcloud.com',
       port=4000,
       user='EnD345nfx9wxmnG.root',
       password='Q4KKSkNgKxF3JIPn'
       # Note: no database specified here for now
    )

def create_database(conn, db_name="choco_crunch"):
    cursor = conn.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name};")
    cursor.execute(f"USE {db_name};")
    cursor.close()

def create_tables(conn):
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS product_info (
        product_code VARCHAR(50) PRIMARY KEY,
        product_name TEXT,
        brand TEXT
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS nutrient_info (
        product_code VARCHAR(50),
        energy_kcal_value FLOAT,
        energy_kj_value FLOAT,
        carbohydrates_value FLOAT,
        sugars_value FLOAT,
        fat_value FLOAT,
        saturated_fat_value FLOAT,
        proteins_value FLOAT,
        fiber_value FLOAT,
        salt_value FLOAT,
        sodium_value FLOAT,
        fruits_vegetables_nuts_estimate_from_ingredients_100g FLOAT,
        nutrition_score_fr INTEGER,
        nova_group INTEGER,
        FOREIGN KEY (product_code) REFERENCES product_info(product_code)
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS derived_metrics (
        product_code VARCHAR(50),
        sugar_to_carb_ratio FLOAT,
        calorie_category TEXT,
        sugar_category TEXT,
        is_ultra_processed TEXT,
        FOREIGN KEY (product_code) REFERENCES product_info(product_code)
    );
    """)
    conn.commit()
    cursor.close()

def clean_nan(df):
    return df.replace({np.nan: None, 'nan': None})

def insert_data(conn, df_product, df_nutrients, df_derived):
    cursor = conn.cursor()
    try:
        product_insert_query = """
            INSERT INTO product_info (product_code, product_name, brand)
            VALUES (%s, %s, %s)
            ON DUPLICATE KEY UPDATE 
                product_name=VALUES(product_name), brand=VALUES(brand);
        """
        product_data = [(row['product_code'], row['product_name'], row['brand']) for _, row in df_product.iterrows()]
        cursor.executemany(product_insert_query, product_data)

        nutrient_insert_query = """
            INSERT INTO nutrient_info (
                product_code, energy_kcal_value, energy_kj_value, carbohydrates_value, sugars_value,
                fat_value, saturated_fat_value, proteins_value, fiber_value, salt_value, sodium_value,
                fruits_vegetables_nuts_estimate_from_ingredients_100g, nutrition_score_fr, nova_group
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                energy_kcal_value=VALUES(energy_kcal_value),
                energy_kj_value=VALUES(energy_kj_value),
                carbohydrates_value=VALUES(carbohydrates_value),
                sugars_value=VALUES(sugars_value),
                fat_value=VALUES(fat_value),
                saturated_fat_value=VALUES(saturated_fat_value),
                proteins_value=VALUES(proteins_value),
                fiber_value=VALUES(fiber_value),
                salt_value=VALUES(salt_value),
                sodium_value=VALUES(sodium_value),
                fruits_vegetables_nuts_estimate_from_ingredients_100g=VALUES(fruits_vegetables_nuts_estimate_from_ingredients_100g),
                nutrition_score_fr=VALUES(nutrition_score_fr),
                nova_group=VALUES(nova_group);
        """
        nutrient_data = [tuple(row) for _, row in df_nutrients.iterrows()]
        cursor.executemany(nutrient_insert_query, nutrient_data)

        derived_insert_query = """
            INSERT INTO derived_metrics (
                product_code, sugar_to_carb_ratio, calorie_category, sugar_category, is_ultra_processed
            ) VALUES (%s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                sugar_to_carb_ratio=VALUES(sugar_to_carb_ratio),
                calorie_category=VALUES(calorie_category),
                sugar_category=VALUES(sugar_category),
                is_ultra_processed=VALUES(is_ultra_processed);
        """
        derived_data = [tuple(row) for _, row in df_derived.iterrows()]
        cursor.executemany(derived_insert_query, derived_data)

        conn.commit()
    except Error as e:
        print(f"Error during insert: {e}")
        conn.rollback()
    finally:
        cursor.close()

# Assuming df1 is your final processed DataFrame from previous code

df_product = df1[['code', 'product_name', 'brands']].drop_duplicates().rename(
    columns={'code':'product_code', 'brands':'brand'}
)
df_nutrients = df1[['code', 'energy-kcal_value','energy-kj_value','carbohydrates_value',
                   'sugars_value', 'fat_value', 'saturated-fat_value', 'proteins_value', 'fiber_value',
                   'salt_value', 'sodium_value', 'fruits-vegetables-nuts-estimate-from-ingredients_100g',
                   'nutrition-score-fr', 'nova-group'
                  ]].rename(columns={
                      'code':'product_code', 'saturated-fat_value':'saturated_fat_value',
                      'nutrition-score-fr':'nutrition_score_fr', 'nova-group':'nova_group'
                  })
df_derived = df1[['code', 'sugar_to_carb_ratio', 'calorie_category', 'sugar_category', 'is_ultra_processed']].rename(
    columns={'code':'product_code'}
)

df_product = clean_nan(df_product)
df_nutrients = clean_nan(df_nutrients)
df_derived = clean_nan(df_derived)

conn = get_connection()
create_database(conn, "choco_crunch")
create_tables(conn)
insert_data(conn, df_product, df_nutrients, df_derived)
conn.close()


In [ ]:
def run_query(conn, query, desc=""):
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    columns = cursor.column_names
    print(f"\n----- {desc} -----")
    print(columns)
    for row in results:
        print(row)
    cursor.close()

conn = get_connection()
conn.database = "choco_crunch"

# --- product_info queries ---
run_query(conn, "SELECT brand, COUNT(*) FROM product_info GROUP BY brand;", "Count products per brand")
run_query(conn, "SELECT brand, COUNT(DISTINCT product_code) FROM product_info GROUP BY brand;", "Unique products per brand")
run_query(conn, "SELECT brand, COUNT(*) AS cnt FROM product_info GROUP BY brand ORDER BY cnt DESC LIMIT 5;", "Top 5 brands by product count")
run_query(conn, "SELECT * FROM product_info WHERE product_name IS NULL OR product_name = '';", "Products with missing product name")
run_query(conn, "SELECT COUNT(DISTINCT brand) FROM product_info;", "Unique brands")
run_query(conn, "SELECT * FROM product_info WHERE product_code LIKE '3%';", "Products with code starting with '3'")

# --- nutrient_info queries ---
run_query(conn, "SELECT product_code, energy_kcal_value FROM nutrient_info ORDER BY energy_kcal_value DESC LIMIT 10;", "Top 10 highest calorie products")
run_query(conn, "SELECT nova_group, AVG(sugars_value) FROM nutrient_info GROUP BY nova_group;", "Average sugars per nova-group")
run_query(conn, "SELECT COUNT(*) FROM nutrient_info WHERE fat_value > 20;", "Count with fat_value > 20g")
run_query(conn, "SELECT AVG(carbohydrates_value) FROM nutrient_info;", "Average carbohydrates_value")
run_query(conn, "SELECT product_code FROM nutrient_info WHERE sodium_value > 1;", "Products with sodium_value > 1g")
run_query(conn, "SELECT COUNT(*) FROM nutrient_info WHERE fruits_vegetables_nuts_estimate_from_ingredients_100g > 0;", "Count products with fruits/vegetables/nuts")
run_query(conn, "SELECT product_code FROM nutrient_info WHERE energy_kcal_value > 500;", "Products with energy-kcal_value > 500")

# --- derived_metrics queries ---
run_query(conn, "SELECT calorie_category, COUNT(*) FROM derived_metrics GROUP BY calorie_category;", "Count per calorie_category")
run_query(conn, "SELECT COUNT(*) FROM derived_metrics WHERE sugar_category = 'High Sugar';", "Count of High Sugar products")
run_query(conn, "SELECT AVG(sugar_to_carb_ratio) FROM derived_metrics WHERE calorie_category = 'High';", "Average sugar_to_carb_ratio for High Calorie")
run_query(conn, "SELECT * FROM derived_metrics WHERE calorie_category = 'High' AND sugar_category = 'High Sugar';", "Products that are both High Calorie and High Sugar")
run_query(conn, "SELECT COUNT(*) FROM derived_metrics WHERE is_ultra_processed = 'Yes';", "Count of ultra-processed products")

# --- Join queries ---
run_query(conn, """
SELECT p.brand, COUNT(*) FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.calorie_category = 'High'
GROUP BY p.brand ORDER BY COUNT(*) DESC LIMIT 5;""", "Top 5 brands with most High Calorie products")

run_query(conn, """
SELECT d.calorie_category, AVG(n.energy_kcal_value)
FROM derived_metrics d
JOIN nutrient_info n ON d.product_code = n.product_code
GROUP BY d.calorie_category;""", "Average energy_kcal_value per calorie_category")

run_query(conn, """
SELECT p.brand, COUNT(*)
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.is_ultra_processed = 'Yes'
GROUP BY p.brand;""", "Count ultra-processed per brand")

run_query(conn, """
SELECT p.brand, p.product_code
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.calorie_category = 'High' AND d.sugar_category = 'High Sugar';""", "Products with High Sugar and High Calorie")

run_query(conn, """
SELECT p.brand, AVG(n.sugars_value)
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
JOIN nutrient_info n ON p.product_code = n.product_code
WHERE d.is_ultra_processed = 'Yes'
GROUP BY p.brand;""", "Average sugars per brand for ultra-processed")

run_query(conn, """
SELECT d.calorie_category, COUNT(*)
FROM derived_metrics d
JOIN nutrient_info n ON d.product_code = n.product_code
WHERE n.fruits_vegetables_nuts_estimate_from_ingredients_100g > 0
GROUP BY d.calorie_category;""", "Number with fruits/veggies/nuts per calorie_category")

run_query(conn, """
SELECT p.product_code, d.sugar_to_carb_ratio, d.calorie_category, d.sugar_category
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
ORDER BY d.sugar_to_carb_ratio DESC LIMIT 5;""", "Top 5 products by sugar_to_carb_ratio")

conn.close()
